In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from types import SimpleNamespace
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.io import wavfile
from scipy.signal import correlate
from scipy import signal as sg
import time
import scipy
from matplotlib import pyplot as plt
import librosa
import sklearn
from sklearn.neural_network import MLPClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data/pitch/test"))

# Any results you write to the current directory are saved as output.

['F5.wav', 'F4.wav', 'F3.wav', 'F2.wav', 'F1.wav', 'M1.wav', 'M2.wav', 'M3.wav', 'M4.wav', 'M5.wav']


In [2]:
class Counters:
    def __init__(self, gross_threshold=0.2):
        self.num_voiced = 0
        self.num_unvoiced = 0
        self.num_voiced_unvoiced = 0
        self.num_unvoiced_voiced = 0
        self.num_voiced_voiced = 0
        self.num_gross_errors = 0
        self.fine_error = 0
        self.e2 = 0
        self.gross_threshold = gross_threshold
        self.nfiles = 0

    def add(self, other):
        if other is not None:
            self.num_voiced += other.num_voiced
            self.num_unvoiced += other.num_unvoiced
            self.num_voiced_unvoiced += other.num_voiced_unvoiced
            self.num_unvoiced_voiced += other.num_unvoiced_voiced
            self.num_voiced_voiced += other.num_voiced_voiced
            self.num_gross_errors += other.num_gross_errors
            self.fine_error += other.fine_error
            self.e2 += other.e2
            self.nfiles += 1

    def __repr__(self):
        nframes = self.num_voiced + self.num_unvoiced
        if self.nfiles > 0:
            self.fine_error /= self.nfiles
        str = [
            f"Num. frames:\t{self.num_unvoiced + self.num_voiced} = {self.num_unvoiced} unvoiced + {self.num_voiced} voiced",
            f"Unvoiced frames as voiced:\t{self.num_unvoiced_voiced}/{self.num_unvoiced} ({100*self.num_unvoiced_voiced/self.num_unvoiced:.2f}%)",
            f"Voiced frames as unvoiced:\t{self.num_voiced_unvoiced}/{self.num_voiced} ({100*self.num_voiced_unvoiced/self.num_voiced:.2f}%)",
            f"Gross voiced errors (>{100*self.gross_threshold}%):\t{self.num_gross_errors}/{self.num_voiced_voiced} ({100*self.num_gross_errors/self.num_voiced_voiced:.2f}%)",
            f"MSE of fine errors:\t{100*self.fine_error:.2f}%",
            f"RMSE:\t{np.sqrt(self.e2/nframes):.2f}"
        ]
        return '\n'.join(str)

In [3]:
def compare(fref, pitch):
    vref = np.loadtxt(fref)
    vtest = np.array(pitch)

    diff_frames = len(vref) - len(vtest)
    if abs(diff_frames) > 5:
        print(f"Error: number of frames in ref ({len(vref)}) != number of frames in test ({len(vtest)})")
        return None
    elif diff_frames > 0:
        vref = np.resize(vref, vtest.shape)
    elif diff_frames < 0:
        vtest = np.resize(vtest, vref.shape)

    counters = Counters()
    counters.num_voiced = np.count_nonzero(vref)
    counters.num_unvoiced = len(vref) - counters.num_voiced
    counters.num_unvoiced_voiced = np.count_nonzero(np.logical_and(vref == 0, vtest != 0))
    counters.num_voiced_unvoiced = np.count_nonzero(np.logical_and(vref != 0, vtest == 0))

    voiced_voiced = np.logical_and(vref != 0, vtest != 0)
    counters.num_voiced_voiced = np.count_nonzero(voiced_voiced)

    f = np.absolute(vref[voiced_voiced] - vtest[voiced_voiced])/vref[voiced_voiced]
    gross_errors = f > counters.gross_threshold
    counters.num_gross_errors = np.count_nonzero(gross_errors)
    fine_errors = np.logical_not(gross_errors)
    counters.fine_error = np.sqrt(np.square(f[fine_errors]).mean())
    counters.e2 = np.square(vref - vtest).sum()

    return counters

In [4]:
def autocorrelation_method(frame, sfreq, threshold=0.55, fmin=40, fmax=400, verbose=False):
    """Estimate pitch using autocorrelation
    """
    # Preprocessing options
    windowing = False
    smoothing_gaussian = False
    smoothing_savgol_filter = True
    
    if verbose:
        plt.figure()
        plt.title('Frame')
        plt.plot(frame)
        plt.xlabel('Samples')
        plt.show
        
    if windowing:
        window = sg.windows.hann(len(frame))
        frame = frame*window
        if verbose:
            plt.figure()
            plt.title('Windowed Frame')
            plt.plot(frame)
            plt.xlabel('Samples')
            plt.show
        
    if smoothing_gaussian:
        std = 0.5
        frame = scipy.ndimage.gaussian_filter1d(frame, std)
        if verbose:
            plt.figure()
            plt.title('Gaussian Smoothed Frame')
            plt.plot(frame)
            plt.xlabel('Samples')
            plt.show
    
    if smoothing_savgol_filter:
        window_lenght = 15
        order = 2
        frame = sg.savgol_filter(frame, window_lenght, order)
        if verbose:
            plt.figure()
            plt.title('Savgol Filter Smoothed Frame')
            plt.plot(frame)
            plt.xlabel('Samples')
            plt.show

    # Calculate autocorrelation using scipy correlate
    frame = frame.astype(np.float)
    frame -= frame.mean()
    amax = np.abs(frame).max()
    if amax > 0:
        frame /= amax
    else:
        return 0

    corr = correlate(frame, frame)
    # keep the positive part
    corr = corr[len(corr)//2:]

    # Find the first minimum
    dcorr = np.diff(corr)
    rmin = np.where(dcorr > 0)[0]
    if len(rmin) > 0:
        rmin1 = rmin[0]
    else:
        return 0

    # Find the next peak
    peak = np.argmax(corr[rmin1:]) + rmin1
    rmax = corr[peak]/corr[0]
    f0 = sfreq / peak

    if rmax > threshold and f0 >= fmin and f0 <= fmax:
        return f0
    else:
        return 0

# SVM-based Voice Activity Detector

## Feature extractor function

In [5]:
def extract_vad_features(audio_file, options): # data is the loaded audio file, fs = sampling frequency
    data, sfreq = librosa.load(audio_file)
    
    # From miliseconds to samples
    ns_windowlength = int(round((options.windowlength * sfreq) / 1000))
    ns_frameshift = int(round((options.frameshift * sfreq) / 1000))
    ns_left_padding = int(round((options.left_padding * sfreq) / 1000))
    ns_right_padding = int(round((options.right_padding * sfreq) / 1000))
    nsamples = len(data)

    zcr = librosa.feature.zero_crossing_rate(data, frame_length=ns_windowlength, hop_length=ns_frameshift)
    rmse = librosa.feature.rmse(y=data, frame_length=ns_windowlength, hop_length=ns_frameshift)
    power = []
    for id, ini in enumerate(range(-ns_left_padding, nsamples - ns_windowlength + ns_right_padding + 1, ns_frameshift)):
        first_sample = max(0, ini)
        last_sample = min(nsamples, ini + ns_windowlength)
        frame = data[first_sample:last_sample]
        power.append(10*np.log10(np.mean([sample**2 for sample in frame])))

    power_np = np.array(power)
    features = np.vstack((zcr, rmse, power_np))
    features = features.T
    
    return features

## Train

In [6]:
def train_svm_vad(options, gui):
    fs = open(options.submission, 'w') if options.submission is not None else None
    totalCounters = Counters()
    print('Extracting features...')
    with open(gui) as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            filename = os.path.join(options.datadir, line + ".wav")
            f0ref_filename = os.path.join(options.datadir, line + ".f0ref")
            
            data, sfreq = librosa.load(filename)
            freqs_ref = np.loadtxt(f0ref_filename)
            
            labels = []
            for a in freqs_ref:
                if int(a)==0:
                    labels.append(0)
                elif int(a)>0:
                    labels.append(1)

            # From miliseconds to samples
            ns_windowlength = int(round((options.windowlength * sfreq) / 1000))
            ns_frameshift = int(round((options.frameshift * sfreq) / 1000))
            ns_left_padding = int(round((options.left_padding * sfreq) / 1000))
            ns_right_padding = int(round((options.right_padding * sfreq) / 1000))
            nsamples = len(data)

            zcr = librosa.feature.zero_crossing_rate(data, frame_length=ns_windowlength, hop_length=ns_frameshift)
            rmse = librosa.feature.rmse(y=data, frame_length=ns_windowlength, hop_length=ns_frameshift)
            power = []
            # mfccs = []
            for id, ini in enumerate(range(-ns_left_padding, nsamples - ns_windowlength + ns_right_padding + 1, ns_frameshift)):
                first_sample = max(0, ini)
                last_sample = min(nsamples, ini + ns_windowlength)
                frame = data[first_sample:last_sample]
                power.append(10*np.log10(np.mean([sample**2 for sample in frame])))
                # mfccs.append(np.array(librosa.feature.mfcc(y=frame, sr=fs)))
                
    power_np = np.array(power)
    features = np.vstack((zcr, rmse, power_np))
    features = features.T

    model = sklearn.svm.SVC(kernel='linear')
    print('model:')
    print(model)
    print('Training...')
    model.fit(features, labels)
    print('Train evaluation...')
    model.score(features, labels)
    print('Mean Accuracy: ', model.score(features, labels))
    return model

In [7]:
start_time = time.time()
fda_ue_options = SimpleNamespace(
    windowlength=32, frameshift=15, left_padding=16, right_padding=16, datadir='data', submission=None)
model_svm = train_svm_vad(fda_ue_options, 'data/pitch/fda_ue.gui')
print(f'Required time: {time.time() - start_time}')

Extracting features...
model:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Training...
Train evaluation...
Mean Accuracy:  0.9138576779026217
Required time: 61.43123197555542


# MLP-based VAD

In [8]:
def train_mlp_vad(options, gui):
    fs = open(options.submission, 'w') if options.submission is not None else None
    totalCounters = Counters()
    print('Extracting features...')
    with open(gui) as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            filename = os.path.join(options.datadir, line + ".wav")
            f0ref_filename = os.path.join(options.datadir, line + ".f0ref")
            
            data, sfreq = librosa.load(filename)
            freqs_ref = np.loadtxt(f0ref_filename)
            
            labels = []
            for a in freqs_ref:
                if int(a)==0:
                    labels.append(0)
                elif int(a)>0:
                    labels.append(1)

            # From miliseconds to samples
            ns_windowlength = int(round((options.windowlength * sfreq) / 1000))
            ns_frameshift = int(round((options.frameshift * sfreq) / 1000))
            ns_left_padding = int(round((options.left_padding * sfreq) / 1000))
            ns_right_padding = int(round((options.right_padding * sfreq) / 1000))
            nsamples = len(data)

            zcr = librosa.feature.zero_crossing_rate(data, frame_length=ns_windowlength, hop_length=ns_frameshift)
            rmse = librosa.feature.rmse(y=data, frame_length=ns_windowlength, hop_length=ns_frameshift)
            power = []
            # mfccs = []
            for id, ini in enumerate(range(-ns_left_padding, nsamples - ns_windowlength + ns_right_padding + 1, ns_frameshift)):
                first_sample = max(0, ini)
                last_sample = min(nsamples, ini + ns_windowlength)
                frame = data[first_sample:last_sample]
                power.append(10*np.log10(np.mean([sample**2 for sample in frame])))
                # mfccs.append(np.array(librosa.feature.mfcc(y=frame, sr=fs)))
                
    power_np = np.array(power)
    features = np.vstack((zcr, rmse, power_np))
    features = features.T

    model = MLPClassifier()
    print('model:')
    print(model)
    print('Training...')
    model.fit(features, labels)
    print('Train evaluation...')
    model.score(features, labels)
    print('Mean Accuracy: ', model.score(features, labels))
    return model

In [9]:
start_time = time.time()
fda_ue_options = SimpleNamespace(
    windowlength=32, frameshift=15, left_padding=16, right_padding=16, datadir='data', submission=None)
model_mlp = train_mlp_vad(fda_ue_options, 'data/pitch/fda_ue.gui')
print(f'Required time: {time.time() - start_time}')

Extracting features...
model:
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)
Training...
Train evaluation...
Mean Accuracy:  0.9138576779026217
Required time: 67.46067690849304


/Users/oscarortigonzalez/Projects/pitch-estimation-notebooks/.venv/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Dataset Validation and Test

In [10]:
def wav2f0(options, gui):
    fs = open(options.submission, 'w') if options.submission is not None else None
    totalCounters = Counters()
    with open(gui) as f:
        if fs is not None:
            print('id,frequency', file=fs)
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            filename = os.path.join(options.datadir, line + ".wav")
            f0ref_filename = os.path.join(options.datadir, line + ".f0ref")
            print("Processing:", filename)
            sfreq, data = wavfile.read(filename)
            nsamples = len(data)
            
            # Get VAD features
            vad_features = extract_vad_features(filename, options)
            vad_results = model_svm.predict(vad_features)
            
            # From miliseconds to samples
            ns_windowlength = int(round((options.windowlength * sfreq) / 1000))
            ns_frameshift = int(round((options.frameshift * sfreq) / 1000))
            ns_left_padding = int(round((options.left_padding * sfreq) / 1000))
            ns_right_padding = int(round((options.right_padding * sfreq) / 1000))
            pitch = []
            i = 0
            for id, ini in enumerate(range(-ns_left_padding, nsamples - ns_windowlength + ns_right_padding + 1, ns_frameshift)):
                # Check if it is a voiced frame
                if vad_results[i] == 1: 
                    first_sample = max(0, ini)
                    last_sample = min(nsamples, ini + ns_windowlength)
                    frame = data[first_sample:last_sample]
                    f0 = autocorrelation_method(frame, sfreq, verbose=False)
                else:
                    f0 = 0
                if fs is not None:
                    print(line + '_' + str(id) + ',', f0, file=fs)
                pitch.append(f0)
                i += 1
                
            # Apply median filter to pitch array
            pitch = sg.medfilt(volume=pitch, kernel_size=3)

            if os.path.isfile(f0ref_filename):
                counters = compare(f0ref_filename, pitch)
                totalCounters.add(counters)


    if totalCounters.num_voiced + totalCounters.num_unvoiced > 0:
        print(f"### Summary")
        print(totalCounters)
        print("-------------------------------\n")

In [11]:
start_time = time.time()
fda_ue_options = SimpleNamespace(
    windowlength=32, frameshift=15, left_padding=16, right_padding=16, datadir='data', submission=None)
wav2f0(fda_ue_options, 'data/pitch/fda_ue.gui')
print(f'Required time: {time.time() - start_time}')

Processing: data/fda_ue/rl001.wav


IndexError: index 100 is out of bounds for axis 0 with size 100

# Make submission

In [ ]:
test_options = SimpleNamespace(
    windowlength=26.5, frameshift=10, left_padding=13.25, right_padding=7, datadir='data/pitch/test', submission='autocorrelation_method_with_SVM_VAD_submission.csv')
wav2f0(test_options, 'data/pitch/test.gui')